In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from Gantry.controller.SNS_layer import R, perceptor, controller_closed_loop_v2, pick_and_place

## Some constant parameters used in simulation.

object_position = [0, 0, -0.34] (m)

target_position = [0.15, 0.15, -0.34] (m)

mass = 0.2 kg

size scaling factor = 0.6

simulation end time = 8 (sec)

perceptor = perceptor (with an inhibitory synapse from the grasping-the-object command neuron to the lifting-the-object-up command neuron. This additional synapse can prevent the controller from transitioning to the lifting up phase too early)

controller = closed loop controller version 2 (jaws use speed control to regulate force)

force threshold gain = 0.01 (equivalent to multiplying force sensory signals by 100. Warning: this method may only work in simulation. Using a small gain in the experiment may lead to a force threshold lower than the noise level.)

In [ ]:
OBJECT_POSITION = [0, 0, -0.34]
TARGET_POSITION = [0.15, 0.15, -0.34]
MASS = 0.2
SIZE_SCALING = 0.6

END_TIME = 8

PERCEPTOR = perceptor
CONTROLLER = controller_closed_loop_v2
FORCE_THRESHOLD_GAIN = 0.01

## Some constant parameters used in plotting.

xlim = (1, 11)

limit for grasper command axis= (0, 0.1) (m)

limit for contact force axis = (0, 50) (N)

In [ ]:
XLIM = (1, 11)
GRASPER_COMMAND_LIM = (0, 0.1)
GRASPER_COMMAND_TICKS = np.arange(0, 0.12, step=0.05)
FORCE_LIM = (0, 50)

## Parameter sweep simulation
Three parameters, namely the time constants, jaw closing speed, and jaw pressure 
may influence the level of contact force if we use the closed loop controller v2.
Below defines the sweep simulation loop and the plot function.

In [ ]:
def parameter_sweep_simulation(grasper_closing_speed_set, tau_set, pressure_set, position_o=OBJECT_POSITION, position_t=TARGET_POSITION, perceptor=PERCEPTOR, controller=CONTROLLER, end_time=END_TIME, force_threshold_gain=FORCE_THRESHOLD_GAIN, zero_time_constant=True, mass=MASS, sizeScaling=SIZE_SCALING):
    
    neuron_history_sweep = []
    sensory_history_sweep = []
    command_history_sweep = []

    for (grasper_closing_speed, tau, pressure) in zip(grasper_closing_speed_set, tau_set, pressure_set):
        perceptor.reset()
        controller.reset()
        perceptor.set_tau(tau)
        neuron_history, sensory_history, command_history = pick_and_place(position_o=position_o, position_t=position_t, perceptor=perceptor, controller=controller, PressureValue=pressure, end_time=end_time, force_threshold_gain=force_threshold_gain, grasper_closing_speed=grasper_closing_speed, zero_time_constant=zero_time_constant, mass=mass, sizeScaling=sizeScaling)
        neuron_history_sweep.append(neuron_history / R)
        sensory_history_sweep.append(sensory_history)
        command_history_sweep.append(command_history)

    neuron_history_sweep = np.array(neuron_history_sweep)
    sensory_history_sweep = np.array(sensory_history_sweep)
    command_history_sweep = np.array(command_history_sweep)

    return neuron_history_sweep, sensory_history_sweep, command_history_sweep


def parameter_sweep_plot(neuron_history_sweep, sensory_history_sweep, command_history_sweep, label_set, xlim=XLIM, grasper_command_lim=GRASPER_COMMAND_LIM, force_lim=FORCE_LIM, grasper_command_ticks=GRASPER_COMMAND_TICKS):
    plt.rcParams['axes.spines.top'] = False
    plt.rcParams['axes.spines.right'] = False
    
    fig, ax = plt.subplots(2)
    time_history = np.arange(neuron_history_sweep[0, :, 0].size) / 240
    for i, label in enumerate(label_set):
        ax[0].plot(time_history, command_history_sweep[i, :, 3], label=label)
        ax[1].plot(time_history, np.sum(sensory_history_sweep[i, :, 3:5], axis=1))
    
    ax[0].set_ylabel('Jaw Command')
    ax[0].set(ylim=grasper_command_lim, xlim=xlim)
    ax[0].spines['bottom'].set_visible(False)
    ax[0].axes.get_xaxis().set_visible(False)
    ax[0].axes.set_yticks(grasper_command_ticks)
    ax[0].legend(frameon=False)
        
    ax[1].set_ylabel('Contact Force')
    ax[1].set(ylim=force_lim, xlim=xlim)
    ax[1].set_xlabel('Time')
    plt.show()
    
    return

## Sweep the closing speed

In [ ]:
grasper_closing_speed_set = np.arange(0.25, 1.5, 0.25)
tau_set = 0.01 * np.ones_like(grasper_closing_speed_set)
pressure_set = 1.25 * np.ones_like(grasper_closing_speed_set)
neuron_history_closed_loop_v2_speed_sweep, sensory_history_closed_loop_v2_speed_sweep, command_history_closed_loop_v2_speed_sweep = parameter_sweep_simulation(grasper_closing_speed_set=grasper_closing_speed_set, tau_set=tau_set, pressure_set=pressure_set)

## Sweep the pressure

In [ ]:
pressure_set = np.array([0.2, 1.0, 1.5, 2.0, 2.5])
grasper_closing_speed_set = 0.5 * np.ones_like(pressure_set)
tau_set = 0.01 * np.ones_like(pressure_set)
neuron_history_closed_loop_v2_pressure_sweep, sensory_history_closed_loop_v2_pressure_sweep, command_history_closed_loop_v2_pressure_sweep = parameter_sweep_simulation(grasper_closing_speed_set=grasper_closing_speed_set, tau_set=tau_set, pressure_set=pressure_set)

## Sweep the time constant

In [ ]:
tau_set = np.arange(0.02, 0.11, 0.02)
grasper_closing_speed_set = 0.5 * np.ones_like(tau_set)
pressure_set = 1.25 * np.ones_like(tau_set)
neuron_history_closed_loop_v2_tau_sweep, sensory_history_closed_loop_v2_tau_sweep, command_history_closed_loop_v2_tau_sweep = parameter_sweep_simulation(grasper_closing_speed_set=grasper_closing_speed_set, tau_set=tau_set, pressure_set=pressure_set)

## Plot parameter sweep results.

In [ ]:
label_set = ['closing speed = {:1.2f}'.format(speed) for speed in np.arange(0.25, 1.5, 0.25)]
parameter_sweep_plot(neuron_history_closed_loop_v2_speed_sweep, sensory_history_closed_loop_v2_speed_sweep, command_history_closed_loop_v2_speed_sweep, label_set)

label_set = ['pressure = {:1.1f}'.format(pressure) for pressure in np.array([0.2, 1.0, 1.5, 2.0, 2.5])]
parameter_sweep_plot(neuron_history_closed_loop_v2_pressure_sweep, sensory_history_closed_loop_v2_pressure_sweep, command_history_closed_loop_v2_pressure_sweep, label_set=label_set)

label_set = ['tau = {:1.2f}'.format(tau) for tau in np.arange(0.02, 0.11, 0.02)]
parameter_sweep_plot(neuron_history_closed_loop_v2_tau_sweep, sensory_history_closed_loop_v2_tau_sweep, command_history_closed_loop_v2_tau_sweep, label_set=label_set)